<br>

### 4.2 Variance Recon

In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.0f}'.format

<br>

#### Restore objects

In [2]:
%store -r gl_pool
%store -r bank_pool
%store -r stats

<br>

#### Comparison Function 

In [3]:
def Compare(x,y,cat):

    combo = pd.merge(x, y, left_index=True, right_index=True).reset_index()  
    combo['match_qty'] = combo[['freq_b','freq_g']].min(axis=1)
        
    combo['xtra_b'] = combo['freq_b'] - combo['freq_g']
    combo['xtra_g'] = combo['freq_g'] - combo['freq_b']
    combo[['xtra_b','xtra_g']] = combo[['xtra_b','xtra_g']].astype(int)
    combo['ext'] = combo['match_qty'] * combo['index']
                
    lefty = pd.merge(x, y, left_index=True, right_index=True, 
                         how='left',indicator=True) \
                         .query('_merge == "left_only"') \
                         .drop('_merge', 1) \
                         .reset_index()
    
    
    righty = pd.merge(x, y, left_index=True, right_index=True, 
                         how='right',indicator=True) \
                         .query('_merge == "right_only"') \
                         .drop('_merge', 1) \
                         .reset_index() 
    
    
    extra_b = combo.loc[combo['xtra_b'] > 0,['index','xtra_b']]
    
    extra_g = combo.loc[combo['xtra_g'] > 0,['index','xtra_g']]
       
    return [combo, lefty, righty, extra_b, extra_g]

<br>
<br>

#### Iterate through Bank / GL data 

In [4]:
cats = ['Credits', 'Checks', 'Other_Pay', 'CL_Adv', 'CL_Pay', 'PR_x']

output = {}

for cat in cats:
    
    outer_list = []
    
    for i in range(0,5):
        bank = list(bank_pool[cat][i]['Amount'])
        gl = list(gl_pool[cat][i]['amount'])
        
        b_count = pd.Series(bank,name='freq_b',dtype='float').round(2).value_counts().to_frame()
        g_count = pd.Series(gl,name='freq_g',dtype='float').round(2).value_counts().to_frame()
        
        view = Compare(b_count,g_count,cat)
        
        res = []
        for tally, df in enumerate(view):
            
            if tally == 0:
                res.append(df['ext'].sum())
                
            if tally == 1:   
                data = pd.Series(np.repeat(df['index'],df['freq_b'])).tolist()
                res.append([0, data])
                
            if tally == 2:
                data = pd.Series(np.repeat(df['index'],df['freq_g'])).tolist()
                res.append([0, data])
            
            if tally > 2:
                res.append(df.values)
        
        outer_list.append(res)
    
    output[cat] = outer_list           

In [5]:
recon = pd.DataFrame(columns=['type',' | ', 'b_list','g_list','extra_b','extra_g'])

for k,v in output.items():
    df = pd.DataFrame(v, columns =[' | ', 'b_list','g_list','extra_b','extra_g'])
    df.insert(0, 'type', k)
    recon = pd.concat([recon,df],axis=0)

recon.rename(index={0: 'Aug', 1: 'Sep', 2: 'Oct', 3: 'Nov', 4: 'Dec'}, inplace=True) 
recon = recon.reset_index().rename(columns={'index': 'month'})

split_b = pd.DataFrame(recon['b_list'].to_list(), columns = ['(b)', 'b_items'])
split_g = pd.DataFrame(recon['g_list'].to_list(), columns = ['(g)', 'g_items'])
recon = pd.concat([recon, split_b, split_g], axis=1) 
recon = recon.drop(['b_list', 'g_list'], axis=1)


cols = [' | ','(b)','(g)']
recon[cols] = recon[cols].astype(float)

reordered = ['month', 'type', ' | ', '(b)', '(g)', 'b_items', 'g_items', 'extra_b', 'extra_g']
recon = recon[reordered]

In [6]:
def Add_Items(x):
    total = sum(x)
    return total

for row in recon.itertuples():
    if len(row[9]) > 0:
        for item in row[9]:
            search_rev = item[0] * -1
            if search_rev in(row[7]):
                row[7].remove(search_rev)
                continue
            else:
                row[7].append(item[0])
                
                
for row in recon.itertuples():
    if len(row[8]) > 0:
        for item in row[8]:
            search_rev = item[0] * -1
            if search_rev in(row[6]):
                row[6].remove(search_rev)
                continue
            else:
                row[6].append(item[0])
                
for row in recon.itertuples():
    if len(row[7]) > 0:
        for item in row[7]:
            search_rev = item * -1
            if search_rev in(row[7]):
                row[7].remove(search_rev)
                row[7].remove(item)
                continue
        
    if len(row[6]) > 0:
        for item in row[6]:
            search_rev = item * -1
            if search_rev in(row[6]):
                row[6].remove(search_rev)
                row[6].remove(item)
                continue

recon['(g)'] = recon['g_items'].apply(Add_Items)
recon['(b)'] = recon['b_items'].apply(Add_Items)

recon = recon.set_index(['month','type'])
recon = recon.merge(stats,left_index=True,right_index=True).reset_index(drop=False)

reordered = ['month', 'type', 'bank', 'gl', ' | ', '(b)', '(g)', 'b_items', 'g_items']
recon = recon[reordered]
%store recon

Stored 'recon' (DataFrame)
